## Escaneado de tablas secuencial y paralelo

Aunque DynamoDB distribuye los elementos en varias particiones físicas, una operación de exploración solo puede leer una partición a la vez. Por este motivo, el rendimiento de una exploración está limitado por el rendimiento máximo de una única partición.

Para maximizar la utilización del aprovisionamiento a nivel de tabla, utilice una Exploración paralela para dividir lógicamente una tabla (o índice secundario) en múltiples segmentos lógicos, y utilice múltiples trabajadores de aplicación para explorar estos segmentos lógicos en paralelo. Cada trabajador de aplicación puede ser un hilo en lenguajes de programación que admitan multihilo o un proceso del sistema operativo. Para obtener más información sobre cómo implementar una exploración paralela, consulte nuestra página de documentación para desarrolladores sobre exploraciones paralelas . La API de Escaneo no es adecuada para todos los patrones de consulta, y para más información sobre por qué los escaneos son menos eficientes que las consultas, por favor lea sobre las implicaciones de rendimiento del Escaneo en nuestra documentación.

El siguiente diagrama muestra cómo una aplicación multihilo realiza una exploración paralela con tres subprocesos de aplicación. La aplicación crea tres subprocesos y cada uno de ellos emite una petición de exploración, explora su segmento designado, recuperando datos de 1 MB cada vez, y devuelve los datos al subproceso principal de la aplicación.


![img](images/image7.jpg)

In [1]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

In [2]:
dt = DynamoTable()
try:
    dt.select_table('logfile')
    print(dt)
except:
    dt.create_table(
        table_name='logfile',
        partition_key='PK',
        partition_key_type='S',
        read_capacity=5,
        write_capacity=5,
        tags=[{'Key': 'workshop-design-patterns', 'Value': 'targeted-for-cleanup'}]
)

Table info:
 - Table name: logfile
 - Table arn: arn:aws:dynamodb:us-east-1:637423169504:table/logfile
 - Table creation: 2024-07-23T13:09:50
 - Key schema: [{'AttributeName': 'PK', 'KeyType': 'HASH'}]
 - Attribute definitions: [{'AttributeName': 'GSI1-PK', 'AttributeType': 'S'}, {'AttributeName': 'PK', 'AttributeType': 'S'}]
 - Point-in-time recovery status: DISABLED
 - Delete protection: False
 - Stream enabled: OFF
 - Tags: [{'Key': 'workshop-design-patterns', 'Value': 'targeted-for-cleanup'}]



En primer lugar, ejecute una Exploración simple (secuencial) para calcular el total de bytes enviados para todos los registros con código de respuesta <> 200. Puede ejecutar una exploración con una expresión de filtro para filtrar los registros no relacionados. A continuación, el trabajador de aplicaciones sumará los valores de todos los registros devueltos en los que el código de respuesta <> 200.

In [5]:
fe = "responsecode <> :f"
eav = {":f": 200}
pageSize = 100

In [6]:
response = dt.table.scan(
                FilterExpression=fe,
                ExpressionAttributeValues=eav,
                Limit=pageSize,
                ProjectionExpression='bytessent'
)

Observe que hay un parámetro Límite establecido en el comando Escanear. Una única operación de Escaneado leerá hasta el número máximo de elementos establecidos (si se utiliza el parámetro Límite) o un máximo de 1 MB de datos, y luego aplicará cualquier filtro a los resultados utilizando FilterExpression. Si el número total de elementos escaneados supera el máximo establecido por el parámetro límite o el límite de tamaño del conjunto de datos de 1 MB, el escaneo se detiene y los resultados se devuelven al usuario como un valor LastEvaluatedKey. Este valor puede utilizarse en una operación posterior para poder continuar donde se dejó.

In [14]:
response['LastEvaluatedKey']

{'PK': '2225'}

In [16]:
totalbytessent = 0

while 'LastEvaluatedKey' in response:
    response = dt.table.scan(
        FilterExpression=fe,
        ExpressionAttributeValues=eav,
        Limit=pageSize,
        ExclusiveStartKey=response['LastEvaluatedKey'],
        ProjectionExpression='bytessent')
    for i in response['Items']:
        totalbytessent += i['bytessent']

In [17]:
totalbytessent

Decimal('23536')

### Ejecutar una exploración paralela

Para realizar una exploración en paralelo, cada trabajador de aplicación emite su propia solicitud de exploración con los siguientes parámetros:

* **Segment**: El segmento a escanear por un trabajador de aplicación específico. Cada trabajador debe utilizar un valor diferente para Segment.
* **TotalSegments**: El número total de segmentos para el escaneo paralelo. Este valor debe ser el mismo que el número de trabajadores que utilizará su aplicación.

Ejectue el siguiente comando:

```bash
python scan_logfile_parallel.py logfile_scan 2
```